# Record

This is how Record utitility is implemented:

In [ ]:
{
    type MyRecord<K extends keyof any, T> = {
        [P in K]: T;
    };
}


toread:

https://stackoverflow.com/questions/65834261/initialize-empty-object-in-typescript-with-record-type

https://stackoverflow.com/questions/70367555/how-to-define-an-empty-object-in-typescript

https://www.google.com/search?q=assign+empty+object+to+Record+site:stackoverflow.com&sxsrf=APwXEddQMWKlTgiIU8RsaneUajlXWL-xzg:1682195166270&sa=X&ved=2ahUKEwjCh9Cqqb7-AhW8kokEHS8JCZYQrQIoBHoECBQQBQ&biw=1453&bih=864&dpr=1


